# Importing Dependencies

In [204]:
import pandas as pd
import re
import plotly.express as px
import plotly.subplots as sp
import warnings
warnings.filterwarnings('ignore')

In [205]:
!pip install --upgrade --no-cache-dir gdown

## Importing Datasets

### OECD Health Statistics Dataset

In [206]:
!gdown 1Tju7UzAW4HJ0idbP7o-743LK0AXhP64g -O 'OECD-Health-Statistics-2022-Frequently-Requested-Data.xls'

Downloading...
From: https://drive.google.com/uc?id=1Tju7UzAW4HJ0idbP7o-743LK0AXhP64g
To: /Users/sun/Desktop/DataScience/Data624/OECD-Health-Statistics-2022-Frequently-Requested-Data.xls
100%|██████████████████████████████████████| 4.17M/4.17M [00:00<00:00, 16.2MB/s]


In [207]:
oecd_health_stats_excel_data = pd.ExcelFile('OECD-Health-Statistics-2022-Frequently-Requested-Data.xls')

In [208]:
# Extracted data from the excel file 'OECD-Health-Statistics-2022-Frequently-Requested-Data.xls' and transformed it into a long format dataframe
data = dict()
data_long = pd.DataFrame()
for n in oecd_health_stats_excel_data.sheet_names:
    print(n)
    data[n] = pd.read_excel(oecd_health_stats_excel_data, sheet_name=n, skiprows=3).replace({'..': None})
    first_col_name = data[n].columns[0]
    long_temp=pd.melt(data[n], id_vars=first_col_name).rename(columns={first_col_name: 'country', 'variable': 'year'}, errors='ignore').dropna()
    long_temp['variable']=n
    long_temp=long_temp[['country', 'year', 'variable', 'value']]
    if len(data_long)==0:
        data_long = long_temp
    else:
        data_long = pd.concat([data_long, long_temp])

Frequently Requested Data
Current expenditure, % GDP
Curr exp, per capita US$PPP
Current exp, per capita, growth
GovtCompIns, % curr exp
GovtCompIns, per capita US$PPP
GovtCompIns, per capita, growth
OOP payments, % curr exp
OOP payments, per capita US$PPP
Pharmaceutical exp, % curr exp
Pharma exp, per capita US$PPP
Physicians
Nurses
Medical graduates
Nursing graduates
Hospital beds
Psychiatric care beds
MRI
CT scanners 
Doctor consultations
MRI exams
CT exams
Inpatient care discharges
ALOS, inpatient care
ALOS, AMI
ALOS, delivery
C-sections
Antibiotics
LE Females at birth
LE Males at birth
LE Total population at birth
LE Females at 65
LE Males at 65
Infant mortality rate
PYLL, All causes, Females
PYLL, All causes, Males 
Suicides, deaths per 100000 pop
Tobacco consumption, females
Tobacco consumption, males
Tobacco consumption, total
Alcohol consumption
Obesity, females (SR)
Obesity, males (SR)
Obesity, total (SR)
Obesity, females (M)
Obesity, males (M)
Obesity, total (M)


### Country Income Classifications Dataset

In [209]:
!gdown 15TQKKCWE0y4FZxbdKHsAlB9Pgb8mwVUY -O 'CountryIncomeClassifications_WB (2023).csv'
# Downloaded from https://datahelpdesk.worldbank.org/knowledgebase/articles/906519-world-bank-country-and-lending-groups

Downloading...
From: https://drive.google.com/uc?id=15TQKKCWE0y4FZxbdKHsAlB9Pgb8mwVUY
To: /Users/sun/Desktop/DataScience/Data624/CountryIncomeClassifications_WB (2023).csv
100%|██████████████████████████████████████| 14.4k/14.4k [00:00<00:00, 17.9MB/s]


In [210]:
country_classifications = pd.read_csv('CountryIncomeClassifications_WB (2023).csv')

# Data Preprocessing and Cleaning
Note: The majority of the code in this section was taken from the code provided by Adam in Lecture 4. Further, the explanations on what each line of code does were mostly taken from Chat GPT. 

## OECD Health Statistics Dataset

In [211]:
# Transformed the 'country' column of the data_long dataframe
  # i.e., Cleans and standardizes the strings in the 'country' column, by removing any special characters, trailing whitespaces, and correcting specific values where necessary
data_long['country']=data_long['country'].str.replace(r"[^A-Za-z\s]", "").str.replace(r"\s+$", "").replace({"Trkiye": "Türkiye"})
  # `data_long['country'].str.replace(r"[^A-Za-z\s]", "")` uses `str.replace` to replace all characters in the 'country' column that are not letters or whitespaces with an empty string. The regular expression r"[^A-Za-z\s]" matches any characters that are not upper or lower case letters or whitespaces.
  # `data_long['country'].str.replace(r"\s+$", "")` uses `str.replace` to remove any whitespaces at the end of each string in the 'country' column. The regular expression r"\s+$" matches one or more whitespaces at the end of each string.
  # `data_long['country'].replace({"Trkiye": "Türkiye"})` uses the replace method to replace the string "Trkiye" with "Türkiye" in the 'country' column.

In [212]:
data_long = data_long.loc[data_long['year'].astype(str).str.len() < 10]
  # Filtered out all rows in the data_long dataframe where where the length of the "year" column converted to a string is greater than or equal to 10

In [213]:
data_long['year']=data_long['year'].astype(str).str.slice(-4).astype(int)
  # Converted the 'year' column from  its original data type to a string and extracts the last 4 characters of each string. The result is then converted to an integer data type.

In [214]:
data_long=data_long.reset_index(drop=True)
  # Reset the index of data_long
  # drop=True is used to avoid the old index being added as a new column

In [215]:
data_long['value']=data_long['value'].astype(float)
  # Converts the 'value' column from its original data type to a float data type

In [216]:
data_long['variable'].value_counts()

Infant mortality rate              2223
LE Females at birth                2152
LE Males at birth                  2149
Alcohol consumption                2135
LE Total population at birth       2117
LE Females at 65                   2054
LE Males at 65                     2051
PYLL, All causes, Females          1961
PYLL, All causes, Males            1961
Suicides, deaths per 100000 pop    1956
Current expenditure, % GDP         1598
Curr exp, per capita US$PPP        1556
GovtCompIns, % curr exp            1530
GovtCompIns, per capita US$PPP     1503
Physicians                         1389
Medical graduates                  1340
ALOS, inpatient care               1270
Pharma exp, per capita US$PPP      1252
Pharmaceutical exp, % curr exp     1228
OOP payments, % curr exp           1197
OOP payments, per capita US$PPP    1190
Inpatient care discharges          1185
Nursing graduates                  1138
Hospital beds                      1100
Doctor consultations                999


In [217]:
def prepare_2d_data(df, years, columns):
    output=df.loc[(df['year'].isin(years)) & df['variable'].isin(columns)]
      # Subsets the original dataframe df to keep only the rows where the year and variable columns are in the lists years and columns respectively, using the .loc indexing method and the isin function.
    output=output.pivot(index=['country', 'year'], columns='variable')
      # Pivots the resulting dataframe so that the variables in the list columns become columns, the country and year columns become the index, and the values in the 'value' column become the values in the cells of the resulting dataframe. The pivot method accomplishes this by grouping the rows by the index specified in the index parameter, and reshaping the values in the specified columns into separate columns.
    output=output.droplevel(0, axis=1)
      # Drops the first level of the columns, which is created after pivoting, using the droplevel method.
    output=output.reset_index()
      # Resets the index of the dataframe using the reset_index method and returns the resulting dataframe as output.
    return output

## Country Income Classifications Dataset

In [218]:
country_classifications.head(2)

,Economy,Code,Region,Income group,Lending category,Other (EMU or HIPC)
0,Aruba,ABW,Latin America & Caribbean,High income,NaN,NaN
1,Afghanistan,AFG,South Asia,Low income,IDA,HIPC


In [219]:
# Dropped all columns except 'Economy' and 'Income group'
country_income_groups = country_classifications[['Economy','Income group']]

In [220]:
# Renamed column 'Economy' to 'country' and 'Income group' to 'income_group'
country_income_groups = country_income_groups.rename(columns={'Economy':'country'})
country_income_groups = country_income_groups.rename(columns={'Income group':'income_group'})
country_income_groups.head(2)

,country,income_group
0,Aruba,High income
1,Afghanistan,Low income


In [221]:
# Kept rows where value in the 'Income group' column was not missing
country_income_groups = country_income_groups[country_income_groups['income_group'].notna()]
country_income_groups.count()

country         217
income_group    217
dtype: int64

## Merging the OECD Health Statistics and Country Income Classification Datasets

In [222]:
# Created a list of unique entries in the 'country' column in data_long
data_long_unique_countries = list((data_long['country'].unique()))
# data_long_unique_countries

In [223]:
# Created a list of unique entries in the 'country' column in country_income_groups
country_income_groups_unique_countries = list((country_income_groups['country'].unique()))
#country_income_groups_unique_countries

In [224]:
# Checked which countries were in data_long but not in country_income_groups then alphebetized the output
dl_butnot_cig_countries = list(set(data_long_unique_countries) - set(country_income_groups_unique_countries))
list.sort(dl_butnot_cig_countries)
#sorted(dl_butnot_cig_countries)

In [225]:
# Checked which countries were in country_income_groups but not in data_long then alphebetized the output
cig_countries_butnot_dl = list(set(country_income_groups_unique_countries) - set(data_long_unique_countries))
list.sort(cig_countries_butnot_dl)
#sorted(cig_countries_butnot_dl)

In [226]:
# Created a dataframe from dl_butnot_cig_countries and cig_countries_butnot_dl to more easily compare the countries in both lists
countries_in_one_dataset_but_not_other = pd.DataFrame({'OECD':pd.Series(dl_butnot_cig_countries),'Country Income Groups':pd.Series(cig_countries_butnot_dl)})
  # Used https://stackoverflow.com/questions/49891200/generate-a-dataframe-from-list-with-different-length

In [227]:
# Converted the dataframe "countries_in_one_dataset_but_not_other" into str object with formatting (so as to display the entire dataframe)
print(countries_in_one_dataset_but_not_other.to_markdown())

|     | OECD         | Country Income Groups          |
|----:|:-------------|:-------------------------------|
|   0 | Korea        | Afghanistan                    |
|   1 | OECD AVERAGE | Albania                        |
|   2 | nan          | Algeria                        |
|   3 | nan          | American Samoa                 |
|   4 | nan          | Andorra                        |
|   5 | nan          | Angola                         |
|   6 | nan          | Antigua and Barbuda            |
|   7 | nan          | Argentina                      |
|   8 | nan          | Armenia                        |
|   9 | nan          | Aruba                          |
|  10 | nan          | Azerbaijan                     |
|  11 | nan          | Bahamas, The                   |
|  12 | nan          | Bahrain                        |
|  13 | nan          | Bangladesh                     |
|  14 | nan          | Barbados                       |
|  15 | nan          | Belarus                  

In [228]:
# Replaced 'Korea, Rep.' in the dataframe "country_income_groups" to 'Korea' to match the "data_long" dataframe
country_income_groups = country_income_groups.replace('Korea, Rep.','Korea')

# Checked that the replacements were successsful
# country_income_groups_unique_countries_v2 = list((country_income_groups['country'].unique()))
# list.sort(country_income_groups_unique_countries_v2)
# print(country_income_groups_unique_countries_v2)

In [229]:
# Performed an left join on data_long and country_income_groups
data_long_country_income_groups = pd.merge(data_long,country_income_groups,on='country',how='left')
data_long_country_income_groups

,country,year,variable,value,income_group
0,Austria,1970,"Current expenditure, % GDP",4.838,High income
1,Belgium,1970,"Current expenditure, % GDP",3.856,High income
2,Canada,1970,"Current expenditure, % GDP",6.353,High income
3,Finland,1970,"Current expenditure, % GDP",4.989,High income
4,France,1970,"Current expenditure, % GDP",5.201,High income
...,...,...,...,...,...
52539,Korea,2020,"Obesity, total (M)",7.400,High income
52540,Latvia,2020,"Obesity, total (M)",23.900,High income
52541,Mexico,2020,"Obesity, total (M)",36.000,Upper middle income
52542,New Zealand,2020,"Obesity, total (M)",31.200,High income


# Generating Visualizations

## **Figure 1**. Out-of-Pocket Healthcare Expenditure per Capita vs. Current Healthcare Expenditure per Capita Across Countries in 2018 and 2020. 

In [230]:
# figure1_2018_2020=prepare_2d_data(data_long, [2018,2020], ['Curr exp, per capita US$PPP', 'OOP payments, per capita US$PPP'])
  # When I tried to run the above line of code, I got the following error, "ValueError: Index contains duplicate entries, cannot reshape"
  # Therefore, I tried to figure out where the duplicate entries were using the code below
# figure1_2018=prepare_2d_data(data_long, [2018], ['Curr exp, per capita US$PPP', 'OOP payments, per capita US$PPP'])
  # I did not get an error with the above line of code
# figure1_2020_currexp=prepare_2d_data(data_long, [2020], ['Curr exp, per capita US$PPP'])
  # I got the same error with the above line of code
# figure1_2020_oop=prepare_2d_data(data_long, [2020], ['OOP payments, per capita US$PPP'])
  # I did not get an error with the above line of code, therefore the duplicated entries are in 'Curr exp, per capita US$PPP'

In [231]:
# Previewed the 2020 entries in 'Curr exp, per capita US$PPP'
mask = (data_long_country_income_groups['year'] == 2020) & (data_long_country_income_groups['variable'] == 'Curr exp, per capita US$PPP')
preview_2020_currexp = data_long_country_income_groups[mask]
preview_2020_currexp = preview_2020_currexp.loc[:, ['country', 'income_group', 'year','variable', 'value']]
print(preview_2020_currexp)

             country         income_group  year                     variable  \
3057       Australia          High income  2020  Curr exp, per capita US$PPP   
3058         Austria          High income  2020  Curr exp, per capita US$PPP   
3059         Belgium          High income  2020  Curr exp, per capita US$PPP   
3060          Canada          High income  2020  Curr exp, per capita US$PPP   
3061           Chile          High income  2020  Curr exp, per capita US$PPP   
...              ...                  ...   ...                          ...   
3149     Switzerland          High income  2020  Curr exp, per capita US$PPP   
3150         Türkiye  Upper middle income  2020  Curr exp, per capita US$PPP   
3151  United Kingdom          High income  2020  Curr exp, per capita US$PPP   
3152   United States          High income  2020  Curr exp, per capita US$PPP   
3153    OECD AVERAGE                  NaN  2020  Curr exp, per capita US$PPP   

             value  
3057   5627.315000

In [232]:
# Displayed duplicated entries in preview_2020_currexp
duplicated_2020_currexp = preview_2020_currexp[preview_2020_currexp.duplicated()]
print(duplicated_2020_currexp)

              country         income_group  year                     variable  \
3115        Australia          High income  2020  Curr exp, per capita US$PPP   
3116          Austria          High income  2020  Curr exp, per capita US$PPP   
3117          Belgium          High income  2020  Curr exp, per capita US$PPP   
3118           Canada          High income  2020  Curr exp, per capita US$PPP   
3119            Chile          High income  2020  Curr exp, per capita US$PPP   
3120         Colombia  Upper middle income  2020  Curr exp, per capita US$PPP   
3121       Costa Rica  Upper middle income  2020  Curr exp, per capita US$PPP   
3122   Czech Republic          High income  2020  Curr exp, per capita US$PPP   
3123          Denmark          High income  2020  Curr exp, per capita US$PPP   
3124          Estonia          High income  2020  Curr exp, per capita US$PPP   
3125          Finland          High income  2020  Curr exp, per capita US$PPP   
3126           France       

In [233]:
# Deleted duplicated 2020 entries in 'Curr exp, per capita US$PPP' from data_long then created fig1_2018_2020 dataframe
fig1_2018_2020_deleting_duplicates = data_long.loc[(data_long['year'].isin([2018, 2020])) & (data_long['variable'].isin(['Curr exp, per capita US$PPP', 'OOP payments, per capita US$PPP']))]
fig1_2018_2020_deleting_duplicates = fig1_2018_2020_deleting_duplicates.drop_duplicates()
fig1_2018_2020 = prepare_2d_data(fig1_2018_2020_deleting_duplicates, [2018, 2020], ['Curr exp, per capita US$PPP', 'OOP payments, per capita US$PPP'])

In [234]:
# Previewed fig1_2018_2020
fig1_2018_2020

variable,country,year,"Curr exp, per capita US$PPP","OOP payments, per capita US$PPP"
0,Australia,2018,5193.845,842.565
1,Australia,2020,5627.315,NaN
2,Austria,2018,5518.538,1016.810
3,Austria,2020,5882.745,988.544
4,Belgium,2018,5315.194,946.897
...,...,...,...,...
72,Türkiye,2020,1304.709,214.372
73,United Kingdom,2018,4189.708,641.852
74,United Kingdom,2020,5018.700,629.229
75,United States,2018,10451.386,1182.653


In [235]:
# Performed an left join on fig1_2018_2020 and country_income_groups
fig1_2018_2020_v2= pd.merge(fig1_2018_2020,country_income_groups,on='country',how='left')
fig1_2018_2020_v2.head(5)

,country,year,"Curr exp, per capita US$PPP","OOP payments, per capita US$PPP",income_group
0,Australia,2018,5193.845,842.565,High income
1,Australia,2020,5627.315,NaN,High income
2,Austria,2018,5518.538,1016.810,High income
3,Austria,2020,5882.745,988.544,High income
4,Belgium,2018,5315.194,946.897,High income


In [236]:
# Created fig1_2018
fig1_2018=prepare_2d_data(data_long, [2018], ['Curr exp, per capita US$PPP', 'OOP payments, per capita US$PPP'])

In [237]:
# Performed an left join on fig1_2018 and country_income_groups
fig1_2018_v2= pd.merge(fig1_2018,country_income_groups,on='country',how='left')
# Drop rows with NaN values in fig1_2018_v2
fig1_2018_v2 = fig1_2018_v2.dropna()
fig1_2018_v2.head(5)

,country,year,"Curr exp, per capita US$PPP","OOP payments, per capita US$PPP",income_group
0,Australia,2018,5193.845,842.565,High income
1,Austria,2018,5518.538,1016.810,High income
2,Belgium,2018,5315.194,946.897,High income
3,Canada,2018,5308.356,807.080,High income
4,Chile,2018,2281.148,758.407,High income


In [238]:
# Deleted duplicated 2020 entries in 'Curr exp, per capita US$PPP' from data_long then created fig1_2020 dataframe
fig1_2020_deleting_duplicates = data_long.loc[(data_long['year'].isin([2020])) & (data_long['variable'].isin(['Curr exp, per capita US$PPP', 'OOP payments, per capita US$PPP']))]
fig1_2020_deleting_duplicates = fig1_2020_deleting_duplicates.drop_duplicates()
fig1_2020 = prepare_2d_data(fig1_2020_deleting_duplicates, [2020], ['Curr exp, per capita US$PPP', 'OOP payments, per capita US$PPP'])

In [239]:
# Performed an left join on fig1_2020 and country_income_groups
fig1_2020_v2= pd.merge(fig1_2020,country_income_groups,on='country',how='left')
# Drop rows with NaN values in fig1_2020_v2
fig1_2020_v2 = fig1_2020_v2.dropna()
fig1_2020_v2.head(5)

,country,year,"Curr exp, per capita US$PPP","OOP payments, per capita US$PPP",income_group
1,Austria,2020,5882.745,988.544,High income
2,Belgium,2020,5406.979,867.304,High income
3,Canada,2020,5828.324,724.500,High income
4,Chile,2020,2412.752,708.795,High income
5,Colombia,2020,1335.877,181.512,Upper middle income


In [240]:
# Created scatterplot with fig1_2018
px.scatter(fig1_2018_v2, x='Curr exp, per capita US$PPP', y='OOP payments, per capita US$PPP', color='income_group')

In [241]:
# Created scatterplot with fig1_2020
px.scatter(fig1_2020_v2, x='Curr exp, per capita US$PPP', y='OOP payments, per capita US$PPP', color='income_group')

In [242]:
# Created side by side subplots with fig1_2018 and fig1_2020
from plotly.subplots import make_subplots
import plotly.graph_objects as go

income_groups = fig1_2018_v2['income_group'].unique()
marker_colors = {group: color for group, color in zip(income_groups, ['#636EFA','#EF553B'])}
                                                                      
fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(x=fig1_2018_v2['Curr exp, per capita US$PPP'], y=fig1_2018_v2['OOP payments, per capita US$PPP'], mode='markers', marker=dict(color=[marker_colors[group] for group in fig1_2018_v2['income_group']])),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=fig1_2020_v2['Curr exp, per capita US$PPP'], y=fig1_2020_v2['OOP payments, per capita US$PPP'], mode='markers', marker=dict(color=[marker_colors[group] for group in fig1_2020_v2['income_group']])),
    row=1, col=2
)

fig.update_layout(height=600, width=1000, title_text="Out-of-Pocket Healthcare Expenditure per Capita vs. Current Healthcare Expenditure per Capita Across Countries in 2018 and 2020")
fig.show()

# Figure 2 Life Expectancy VS Current Healthcare Expenditure per Capita and Alcohol Consumption per Population 15+ by Country in 2018 and 2020 

In [243]:
continents={'Austria' : 'Europe', 'Belgium': 'Europe', 'Canada': 'N. America', 'Finland': 'Europe', 
            'France': 'Europe', 'Germany': 'Europe', 'Iceland': 'Europe', 'Ireland': 'Europe', 'Japan': 'Asia', 
            'Korea': 'Asia', 'New Zealand': 'Oceania', 'Norway': 'Europe', 'Portugal': 'Europe', 'Spain': 'Europe', 
            'Sweden': 'Europe', 'Switzerland': 'Europe', 'United Kingdom': 'Europe', 'United States': 'N. America',
            'Australia': 'Oceania', 'Denmark': 'Europe', 'Netherlands': 'Europe', 'Israel': 'Asia', 'Türkiye': 'Asia',
            'Luxembourg': 'Europe', 'Greece': 'Europe', 'Italy': 'Europe', 'Czech Republic': 'Europe', 
            'Poland': 'Europe', 'Hungary': 'Europe', 'Slovak Republic': 'Europe', 'Estonia': 'Europe', 
            'Mexico': 'N. America', 'Chile': 'S. America', 'Colombia': 'S. America', 'Costa Rica': 'N. America', 
            'Latvia': 'Europe', 'Lithuania': 'Europe', 'Slovenia': 'Europe', 'OECD AVERAGE': 'OECD'}

regions={'Austria' : 'W. Europe', 'Belgium': 'W. Europe', 'Canada': 'N. America', 'Finland': 'N. Europe', 
            'France': 'W. Europe', 'Germany': 'W. Europe', 'Iceland': 'N. Europe', 'Ireland': 'N. Europe', 'Japan': 'E.  Asia', 
            'Korea': 'E.  Asia', 'New Zealand': 'Oceania', 'Norway': 'N. Europe', 'Portugal': 'S. Europe', 'Spain': 'S. Europe', 
            'Sweden': 'N. Europe', 'Switzerland': 'W. Europe', 'United Kingdom': 'N. Europe', 'United States': 'N. America',
            'Australia': 'Oceania', 'Denmark': 'N. Europe', 'Netherlands': 'W. Europe', 'Israel': 'W. Asia', 'Türkiye': 'W. Asia',
            'Luxembourg': 'W. Europe', 'Greece': 'S. Europe', 'Italy': 'S. Europe', 'Czech Republic': 'E.  Europe', 
            'Poland': 'E.  Europe', 'Hungary': 'E.  Europe', 'Slovak Republic': 'E.  Europe', 'Estonia': 'E.  Europe', 
            'Mexico': 'Latin America', 'Chile': 'Latin America', 'Colombia': 'Latin America', 'Costa Rica': 'Latin America', 
            'Latvia': 'E.  Europe', 'Lithuania': 'E.  Europe', 'Slovenia': 'E.  Europe', 'OECD AVERAGE': 'OECD'}

In [244]:
# data_long['region']=data_long['country'].replace(continents)
data_long_f2 = data_long.copy()
data_long_f2['region']=data_long_f2['country'].replace(continents)
data_long_2018 = data_long_f2[data_long_f2["year"] == 2018 ]
data_long_2018= pd.merge(data_long_2018,country_income_groups,on='country',how='left')
data_long_2018


,country,year,variable,value,region,income_group
0,Australia,2018,"Current expenditure, % GDP",10.074,Oceania,High income
1,Austria,2018,"Current expenditure, % GDP",10.345,Europe,High income
2,Belgium,2018,"Current expenditure, % GDP",10.788,Europe,High income
3,Canada,2018,"Current expenditure, % GDP",10.846,N. America,High income
4,Chile,2018,"Current expenditure, % GDP",9.159,S. America,High income
...,...,...,...,...,...,...
1411,Korea,2018,"Obesity, total (M)",5.900,Asia,High income
1412,Latvia,2018,"Obesity, total (M)",24.100,Europe,High income
1413,Mexico,2018,"Obesity, total (M)",36.100,N. America,Upper middle income
1414,New Zealand,2018,"Obesity, total (M)",32.500,Oceania,High income


In [245]:
data_long_2018_healthExpenditure = data_long_2018[data_long_2018["variable"] == "OOP payments, per capita US$PPP"]
data_long_2018_AlcoholConsumption = data_long_2018[data_long_2018["variable"] == "Alcohol consumption"]
data_long_2018_LE = data_long_2018[data_long_2018["variable"] == "LE Total population at birth"]

data_long_2018_healthExpenditure

,country,year,variable,value,region,income_group
266,Australia,2018,"OOP payments, per capita US$PPP",842.565,Oceania,High income
267,Austria,2018,"OOP payments, per capita US$PPP",1016.810,Europe,High income
268,Belgium,2018,"OOP payments, per capita US$PPP",946.897,Europe,High income
269,Canada,2018,"OOP payments, per capita US$PPP",807.080,N. America,High income
270,Chile,2018,"OOP payments, per capita US$PPP",758.407,S. America,High income
271,Colombia,2018,"OOP payments, per capita US$PPP",180.341,S. America,Upper middle income
272,Costa Rica,2018,"OOP payments, per capita US$PPP",342.909,N. America,Upper middle income
273,Czech Republic,2018,"OOP payments, per capita US$PPP",443.834,Europe,High income
274,Denmark,2018,"OOP payments, per capita US$PPP",733.603,Europe,High income
275,Estonia,2018,"OOP payments, per capita US$PPP",579.508,Europe,High income


In [246]:
column_names_2018_f2 = ["country","year","LE Total population at birth","region","income_group","Alcohol consumption","OOP payments, per capita US$PPP"]
merged_df_2018_f2 = data_long_2018_LE.merge(
                            data_long_2018_AlcoholConsumption, on=['country','year','region','income_group']
                            ).merge(data_long_2018_healthExpenditure, on=['country','year','region','income_group'])
merged_df_2018_f2.drop(columns=["variable_x","variable_y","variable"], inplace=True)
merged_df_2018_f2.columns = column_names_2018_f2
merged_df_2018_f2

,country,year,LE Total population at birth,region,income_group,Alcohol consumption,"OOP payments, per capita US$PPP"
0,Austria,2018,81.800000,Europe,High income,11.3,1016.810
1,Belgium,2018,81.700000,Europe,High income,9.2,946.897
2,Canada,2018,81.900000,N. America,High income,8.2,807.080
3,Chile,2018,80.400000,S. America,High income,6.3,758.407
4,Colombia,2018,76.500000,S. America,Upper middle income,4.2,180.341
5,Costa Rica,2018,80.300000,N. America,Upper middle income,3.3,342.909
6,Czech Republic,2018,79.100000,Europe,High income,11.8,443.834
7,Denmark,2018,81.000000,Europe,High income,9.7,733.603
8,Estonia,2018,78.500000,Europe,High income,10.0,579.508
9,Finland,2018,81.800000,Europe,High income,8.4,795.477


In [251]:
data_long_2020 = data_long_f2[data_long_f2["year"] == 2020]
data_long_2020= pd.merge(data_long_2020,country_income_groups,on='country',how='left')
data_long_2020_healthExpenditure = data_long_2020[data_long_2020["variable"] == "OOP payments, per capita US$PPP"]
data_long_2020_AlcoholConsumption = data_long_2020[data_long_2020["variable"] == "Alcohol consumption"]
data_long_2020_LE = data_long_2020[data_long_2020["variable"] == "LE Total population at birth"]


In [252]:
column_names_2020_f2 = ["country","year","LE Total population at birth","region","income_group","Alcohol consumption","OOP payments, per capita US$PPP"]
merged_df_2020_f2 = data_long_2020_LE.merge(
                            data_long_2020_AlcoholConsumption, on=['country','year','region','income_group']
                            ).merge(data_long_2020_healthExpenditure, on=['country','year','region','income_group'])
merged_df_2020_f2.drop(columns=["variable_x","variable_y","variable"], inplace=True)
merged_df_2020_f2.columns = column_names_2020_f2
merged_df_2020_f2

,country,year,LE Total population at birth,region,income_group,Alcohol consumption,"OOP payments, per capita US$PPP"
0,Austria,2020,81.3,Europe,High income,11.3,988.544
1,Canada,2020,81.7,N. America,High income,8.1,724.500
2,Costa Rica,2020,80.6,N. America,Upper middle income,3.2,328.204
3,Czech Republic,2020,78.3,Europe,High income,11.6,438.546
4,Denmark,2020,81.6,Europe,High income,9.7,730.470
5,Estonia,2020,78.9,Europe,High income,10.5,585.209
6,Finland,2020,82.0,Europe,High income,8.2,755.611
7,France,2020,82.3,Europe,High income,10.4,488.895
8,Hungary,2020,75.7,Europe,High income,10.4,613.075
9,Iceland,2020,83.1,Europe,High income,7.4,693.759


In [ ]:
# merged_df_2020_f2 = merged_df_2020_f2.loc[~(merged_df_2020_f2.region=='E. Asia')]
# print(merged_df_2020_f2)
merged_df_2020_f2 = merged_df_2020_f2.loc[(merged_df_2020_f2['region']=='Asia')&(merged_df_2020_f2['income_group']=='High income')]
print(merged_df_2020_f2)
merged_df_2018_f2 = merged_df_2018_f2.loc[(merged_df_2018_f2['region']=='Asia')&(merged_df_2018_f2['income_group']=='High income')]
print(merged_df_2018_f2)

   country  year  LE Total population at birth region income_group  \
11   Japan  2020                          84.7   Asia  High income   
12   Korea  2020                          83.5   Asia  High income   

    Alcohol consumption  OOP payments, per capita US$PPP  
11                  6.7                          621.042  
12                  7.9                          994.077  
   country  year  LE Total population at birth region income_group  \
16  Israel  2018                     82.916296   Asia  High income   
18   Japan  2018                     84.300000   Asia  High income   
19   Korea  2018                     82.700000   Asia  High income   

    Alcohol consumption  OOP payments, per capita US$PPP  
16                  3.0                          600.292  
18                  7.2                          591.808  
19                  8.5                          992.747  


In [ ]:
fig =px.scatter(merged_df_2018_f2.dropna(), x='OOP payments, per capita US$PPP', 
            y='LE Total population at birth', size='Alcohol consumption', hover_name='country', color='Alcohol consumption',text='country')

fig.update_traces(textposition='top center')
fig.show()

In [ ]:
px.scatter(merged_df_2020_f2.dropna(), x='OOP payments, per capita US$PPP', 
            y='LE Total population at birth', size='Alcohol consumption', hover_name='country', color='Alcohol consumption')

# Figure 3. Densities of Different Healthcare Resource per 1000 Population by Country

In [256]:
data_long_f2_2018 = data_long_f2[data_long_f2['year']==2018]
data_long_f2_2020 = data_long_f2[data_long_f2['year']==2020]
data_long_f2_2018= pd.merge(data_long_f2_2018,country_income_groups,on='country',how='left')
data_long_f2_2020= pd.merge(data_long_f2_2020,country_income_groups,on='country',how='left')

data_long_f2_2018

,country,year,variable,value,region,income_group
0,Australia,2018,"Current expenditure, % GDP",10.074,Oceania,High income
1,Austria,2018,"Current expenditure, % GDP",10.345,Europe,High income
2,Belgium,2018,"Current expenditure, % GDP",10.788,Europe,High income
3,Canada,2018,"Current expenditure, % GDP",10.846,N. America,High income
4,Chile,2018,"Current expenditure, % GDP",9.159,S. America,High income
...,...,...,...,...,...,...
1411,Korea,2018,"Obesity, total (M)",5.900,Asia,High income
1412,Latvia,2018,"Obesity, total (M)",24.100,Europe,High income
1413,Mexico,2018,"Obesity, total (M)",36.100,N. America,Upper middle income
1414,New Zealand,2018,"Obesity, total (M)",32.500,Oceania,High income


In [ ]:
# Pick Mexico and Canada as samples
column_picked = ["Hospital beds","CT scanners ","MRI","Nurses","Physicians"]
# density_2018 = data_long_country_income_groups_2018.loc[data_long_country_income_groups_2018.country.isin(['Mexico','Canada'])]
# density_2018 = density_2018.loc[density_2018.variable.isin(column_picked)]
# density_2020 = data_long_country_income_groups_2020.loc[data_long_country_income_groups_2020.country.isin(['Mexico','Canada'])]
# density_2020 = density_2020.loc[density_2020.variable.isin(column_picked)]
# density_2018


In [259]:
data_long_f2

,country,year,variable,value,region
0,Austria,1970,"Current expenditure, % GDP",4.838,Europe
1,Belgium,1970,"Current expenditure, % GDP",3.856,Europe
2,Canada,1970,"Current expenditure, % GDP",6.353,N. America
3,Finland,1970,"Current expenditure, % GDP",4.989,Europe
4,France,1970,"Current expenditure, % GDP",5.201,Europe
...,...,...,...,...,...
52539,Korea,2020,"Obesity, total (M)",7.400,Asia
52540,Latvia,2020,"Obesity, total (M)",23.900,Europe
52541,Mexico,2020,"Obesity, total (M)",36.000,N. America
52542,New Zealand,2020,"Obesity, total (M)",31.200,Oceania


In [258]:
data_long_f2_2018

,country,year,variable,value,region,income_group
0,Australia,2018,"Current expenditure, % GDP",10.074,Oceania,High income
1,Austria,2018,"Current expenditure, % GDP",10.345,Europe,High income
2,Belgium,2018,"Current expenditure, % GDP",10.788,Europe,High income
3,Canada,2018,"Current expenditure, % GDP",10.846,N. America,High income
4,Chile,2018,"Current expenditure, % GDP",9.159,S. America,High income
...,...,...,...,...,...,...
1411,Korea,2018,"Obesity, total (M)",5.900,Asia,High income
1412,Latvia,2018,"Obesity, total (M)",24.100,Europe,High income
1413,Mexico,2018,"Obesity, total (M)",36.100,N. America,Upper middle income
1414,New Zealand,2018,"Obesity, total (M)",32.500,Oceania,High income


In [264]:
f3_2018=prepare_2d_data(data_long, [2018], column_picked)
f3_2018 = f3_2018.merge(data_long_f2_2018[['country','income_group','region']], on='country').drop_duplicates().reset_index(drop=True)

f3_2020=prepare_2d_data(data_long, [2020], column_picked)
f3_2020 = f3_2020.merge(data_long_f2_2020[['country','income_group','region']], on='country').drop_duplicates().reset_index(drop=True)
f3_2020


,country,year,CT scanners,Hospital beds,MRI,Nurses,Physicians,income_group,region
0,Australia,2020,67.68,NaN,14.79,12.26,3.90,High income,Oceania
1,Austria,2020,28.49,7.05,25.35,10.48,5.35,High income,Europe
2,Belgium,2020,24.01,5.53,11.44,NaN,3.21,High income,Europe
3,Canada,2020,NaN,2.55,NaN,10.06,2.73,High income,N. America
4,Chile,2020,NaN,2.01,NaN,3.47,2.79,High income,S. America
5,Colombia,2020,5.93,1.69,NaN,1.53,2.38,Upper middle income,S. America
6,Costa Rica,2020,2.93,1.15,0.39,NaN,NaN,Upper middle income,N. America
7,Czech Republic,2020,16.26,6.50,11.03,8.66,4.10,High income,Europe
8,Denmark,2020,40.64,2.59,NaN,NaN,NaN,High income,Europe
9,Estonia,2020,20.31,4.46,15.04,6.38,3.48,High income,Europe


In [275]:
# Rescale
for c in column_picked:
    f3_2018[c]= f3_2018[c]/f3_2018[c].max()
f3_2018=f3_2018.dropna()
f3_2018.head()

,country,year,CT scanners,Hospital beds,MRI,Nurses,Physicians,income_group,region
1,Austria,2018,0.598340,0.584405,0.599949,0.387916,0.859016,High income,Europe
2,Belgium,2018,0.495643,0.451768,0.296787,0.625071,0.513115,High income,Europe
5,Colombia,2018,0.128216,0.137460,0.006119,0.079616,0.362295,Upper middle income,S. America
7,Czech Republic,2018,0.333817,0.532154,0.263896,0.481084,0.662295,High income,Europe
9,Estonia,2018,0.392324,0.364148,0.347272,0.355167,0.570492,High income,Europe


In [276]:
# Rescale
for c in column_picked:
    f3_2020[c]= f3_2020[c]/f3_2020[c].max()
f3_2020=f3_2020.dropna()
f3_2020.head()

,country,year,CT scanners,Hospital beds,MRI,Nurses,Physicians,income_group,region
1,Austria,2020,0.246240,0.557312,0.441715,0.570495,0.974499,High income,Europe
7,Czech Republic,2020,0.140536,0.513834,0.192194,0.471421,0.746812,High income,Europe
9,Estonia,2020,0.175540,0.352569,0.262067,0.347305,0.633880,High income,Europe
11,France,2020,0.163613,0.452964,0.283325,0.615678,0.577413,High income,Europe
14,Hungary,2020,0.083319,0.534387,0.085729,0.358193,0.571949,High income,Europe


In [278]:
radar=pd.melt(f3_2018[(f3_2018['region']=='Asia')&(f3_2018['income_group']=='High income')], id_vars=['country', 'year','income_group'])
fig3_2018 = px.line_polar(radar, r='value', theta='variable', line_close=True, range_r=[0, 1], color='country')
fig3_2018.update_traces(fill='toself', opacity=0.5)
fig3_2018.update_layout(title='Indicator Values for High-Income Asian Countries in 2018')

fig3_2018.show()

In [274]:
radar=pd.melt(f3_2020.loc[(f3_2020['region']=='Asia')&(f3_2020['income_group']=='High income')], id_vars=['country', 'year','income_group'])
fig3_2020 = px.line_polar(radar, r='value', theta='variable', line_close=True, range_r=[0, 1], color='country')
fig3_2020.update_traces(fill='toself', opacity=0.5)
fig3_2020.show()